In [1]:
#import dependencies 
import pandas as pd
import numpy as np
import json 

In [3]:
#import data files 
data_file = 'purchase_data.json'
purchase_data = pd.read_json(data_file, orient="records")
purchase_data.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


# Player Counts & Demographics

In [5]:
# obtain player demographics 
player_demos = purchase_data.loc[:, ["Gender", "SN", "Age"]]
player_demos.head()

,Gender,SN,Age
0,Male,Aelalis34,38
1,Male,Eolo46,21
2,Male,Assastnya25,34
3,Male,Pheusrical25,21
4,Male,Aela59,23


In [7]:
# obtain unique values of player demographics to obtain total players
player_demos = player_demos.drop_duplicates()
total_players = player_demos.count()[0]
total_players
print("Total players: " + str(total_players))

Total players: 573


Total players: SN    573
dtype: int64


In [ ]:
# Number of unique items 


In [ ]:
# Total number of purchases 
total_purchases = data_df.groupby("Item Name").agg({"Price": np.sum, "Item Name": pd.Series.nunique})
total_purchases

item_count = total_purchases.count()
item_count 

In [ ]:
# Total number of 